In [68]:
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from PIL import Image
from skimage import io, transform
import time
import torch
import torch.nn as nn
from torchvision import datasets, models, transforms

In [69]:
# Configurations

# Device
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

# Num epochs
num_epochs = 5

# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Batch size
batch_size = 32

# Data set
train_path = '/lab/vislab/DATA/CUB/images'

# Loss function
criterion = nn.TripletMarginLoss()

# Model 
model = models.resnet18()

In [79]:
# Data Loader

# Transform
transform =  transforms.Compose(
            [transforms.Resize([256, 256]),
                transforms.RandomCrop(224),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])])

train_data = datasets.ImageFolder(train_path, transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4)

In [84]:
def train_model():
    """generic function to train model"""
    
    optimizer.zero_grad()
    train_loss = []
    start_time = datetime.now()
    
    for epoch in range(num_epochs): 
        print("epoch num:", epoch)
        model.train()
        
        for idx, (imgs, labels) in enumerate(train_loader): 
            imgs, labels = imgs.to(device), labels.to(device)
            output = model.forward(imgs)

            loss = criterion(output, labels)
            loss = Variable(loss, requires_grad = True)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss)
            
        
        time_elapsed = datetime.now() - start_time 
        print('Time elapsed (hh:mm:ss.ms) {}'.format(time_elapsed))
        
    return train_loss
    

In [ ]:
# Run Script
model.to(device)

loss = train_model()
print(loss[0], loss[-1])

epoch num: 0


In [ ]:
# Visualization Script